# 1.2: Missing Values
After pre-processing the data, there are still a number of variables with missing values:

In [1]:
import pandas as pd
import numpy as np
import typing
from master.nodes.ETL import summarise

In [2]:
# Load preprocessed data
%reload_kedro
reviews = io.load('reviews_preproc'); summarise(reviews)

2019-07-06 07:20:52,317 - root - INFO - ** Kedro project TWA Assignment
2019-07-06 07:20:52,320 - anyconfig - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/TWA/MBUSA_Text_Web_Analytics/conf/base/logging.yml
2019-07-06 07:20:52,343 - anyconfig - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/TWA/MBUSA_Text_Web_Analytics/conf/base/catalog.yml
2019-07-06 07:20:52,350 - anyconfig - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/TWA/MBUSA_Text_Web_Analytics/conf/base/credentials.yml
2019-07-06 07:20:52,353 - anyconfig - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/TWA/MBUSA_Text_Web_Analytics/conf/base/parameters.yml
2019-07-06 07:20:52,356 - root - INFO - Defined global variables proj_dir, proj_name, conf and io
2019-07-06 07:20:52,358 - kedro.io.data_catalog - INFO - Loading data from `reviews_preproc` (PickleLocalDataSet)..

,dtype,n_null,n_valid,unique,mean,std,min,25%,50%,75%,max,skew
product_id,int64,0,23486,1206,918.119,203.299,0.0,861.0,936.0,1078.0,1205.0,-2.088
author_age,int64,0,23486,77,43.199,12.280,18.0,34.0,41.0,52.0,99.0,0.526
review_title,object,3810,19676,13994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
review_text,object,845,22641,22635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
star_rating,int64,0,23486,5,4.196,1.110,1.0,4.0,5.0,5.0,5.0,-1.314
recommend_flag,int64,0,23486,2,0.822,0.382,0.0,1.0,1.0,1.0,1.0,-1.687
upvotes,int64,0,23486,82,2.536,5.702,0.0,0.0,1.0,3.0,122.0,6.473
product_category_division,category,14,23472,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_category_department,category,14,23472,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_category_class,category,14,23472,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- The `Title` column contains 3810 missing values, so it appears that titles are optional.
- The `Review Text` contains 845 missing values, which we'll need to exclude because our project task is to extract information from these reviews.

We only need to exclude these missing rows if *both* `Title` and `Review Text` are missing. There is one review that contains only a title and 2966 reviews that contain only body text:

In [3]:
print("Number of reviews with a title but no body text:",
      len(reviews[
          reviews.review_text.isnull() &\
          reviews.review_title.notnull()
      ])
 )
print("Number of reviews with body text but no title:",
      len(reviews[
          reviews.review_text.notnull() &\
          reviews.review_title.isnull()
      ])
)

Number of reviews with a title but no body text: 1
Number of reviews with body text but no title: 2966


In [4]:
def text_fields(df: pd.DataFrame) -> list:
    """Returns the list of text fields in the DataFrame."""
    return df.select_dtypes("object")

text_fields(reviews).head()

,review_title,review_text
id,,
0,NaN,Absolutely wonderful - silky and sexy and comf...
1,NaN,Love this dress! it's sooo pretty. i happene...
2,Some major design flaws,I had such high hopes for this dress and reall...
3,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl..."
4,Flattering shirt,This shirt is very flattering to all due to th...


In [5]:
def drop_missing_text(df):
    """Drop any reviews that are missing entries for all text fields."""
    return df.dropna(subset=text_fields(df).columns, how="all")

reviews_clean = drop_missing_text(reviews); summarise(reviews_clean)

,dtype,n_null,n_valid,unique,mean,std,min,25%,50%,75%,max,skew
product_id,int64,0,22642,1179,919.340,202.266,1.0,861.0,936.0,1078.0,1205.0,-2.091
author_age,int64,0,22642,77,43.280,12.327,18.0,34.0,41.0,52.0,99.0,0.516
review_title,object,2966,19676,13994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
review_text,object,1,22641,22635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
star_rating,int64,0,22642,5,4.184,1.116,1.0,4.0,5.0,5.0,5.0,-1.288
recommend_flag,int64,0,22642,2,0.819,0.385,0.0,1.0,1.0,1.0,1.0,-1.656
upvotes,int64,0,22642,82,2.630,5.786,0.0,0.0,1.0,3.0,122.0,6.381
product_category_division,category,13,22629,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_category_department,category,13,22629,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_category_class,category,13,22629,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


After removing the training examples with missing text fields, there are still 13 reviews that are missing  category labels (`product_category_division`, `product_category_department` and `product_category_class`). We have three options for handling these categories:
1. **Exclusion**: Remove the reviews from the dataset.
2. **Imputation**: Impute the missing values using patterns in the existing dataset.
3. **Categorisation**: Add "missing" as another factor level for each of the category variables.

The uncategorised reviews only correspond to 7 unique products. This is a sufficiently small number, so we'll attempt to manually impute the missing categories (option #2).

In [6]:
def category_list(df: pd.DataFrame) -> list:
    """Returns the list of category fields in the DataFrame."""
    return list(df.filter(like="category").columns)
    
def uncategorised(df):
    """Returns the reviews that do not have product categories."""
    return df[df.filter(category_list(df)).isnull().any(1)]

print("Categories:", category_list(reviews))
WORD = "hoodie"
print("Number of uncategorised reviews with missing categories:", len(uncategorised(reviews_clean)))
uncat_products = uncategorised(reviews_clean).product_id.unique()
print("Number of uncategorised products with missing categories:", len(uncat_products))
print("Unique IDs of uncategorised products: {}".format(uncat_products))

Categories: ['product_category_division', 'product_category_department', 'product_category_class']
Number of uncategorised reviews with missing categories: 13
Number of uncategorised products with missing categories: 7
Unique IDs of uncategorised products: [ 72 492 152 184 772 665 136]


By inspecting the corresponding reviews for the uncategorised products, we can infer some of the product types from the nouns used in the reviews. To impute the missing product categories, we will:
1. Create a dictionary of keywords to represent our best guess of the type of each uncategorised product.
2. Using these keywords, find the other reviews that also contain this keyword.
3. Across each of these similar reviews, identify the most frequent category hierarchy and apply it to the uncategorised reviews.

In [7]:
# Inspect the review title and body text for uncategorised products
reviews[reviews.product_id.isin(uncat_products)][["product_id","review_title", "review_text"]]

,product_id,review_title,review_text
id,,,
9444,72,My favorite socks!!!,"I never write reviews, but these socks are so ..."
13767,492,So soft!,I just love this hoodie! it is so soft and com...
13768,492,Wardrobe staple,Love this hoodie. so soft and goes with everyt...
13787,492,NaN,NaN
16216,152,Warm and cozy,"Just what i was looking for. soft, cozy and warm."
16221,152,Love!,I am loving these. they are quite long but are...
16223,152,"""long and warm""",These leg warmers are perfect for me. they are...
18626,184,Nubby footless tights,"These are amazing quality. i agree, size up to..."
18671,184,New workhorse,These tights are amazing! if i care for them w...


From the table above, we can infer some of the product types:
- 72: socks
- 492: hoodie
- 152: leg warmers
- 184: tights
- 772: sweatshirt
- 665: ???
- 136: socks

Let's take a closer look at the review for product 665...

In [8]:
reviews.loc[21532].review_text

'Got these on sale...absolutely love eberjey! fabric is super soft and easy to wash. more "normal" than other eberjey styles (not lingerie). i\'m 5\'6", 125 lbs and small fit well. i am small busted and there is definitely extra room but doesn\'t make it fit weird at all. highly recommend!'

For this product, a reasonable guess might be some kind of underwear, since the noun *lingerie* is mentioned as a comparison to the product ("not lingerie"). The customer also describes themselves as *small busted*, so the product is likely to be a bra or some kind of intimate product. We'll use *underwear* as our best guess:
- 72: socks
- 492: hoodie
- 152: leg warmers
- 184: tights
- 772: sweatshirt
- 665: underwear
- 136: socks

Using these keywords, let's create a function that identifies the most likely category for each product. First, we'll define a search function to search the text fields of each review:

In [9]:
def search(df: pd.DataFrame, term: str) -> pd.DataFrame:
    """Searches the review text and title fields for a token string.

    Args:
        df: the DataFrame in which to run the seaarch.
        term: the keyword used to filter the reviews.

    Returns:
        A DataFrame with the subset of reviews that match the search.
        If no rows match the search, returns None.
    """
    matches = text_fields(df).apply(lambda x: x.str.contains(term)).any(1)
    if any(matches):
        return df.loc[matches]
    else:
        return None

In [10]:
# Example application: change the value of WORD to search different terms
WORD = "hoodie"
results = search(reviews, WORD)
print("Number of reviews containing '{}': {}".format(WORD, len(results)))
text_fields(results).head()

Number of reviews containing 'hoodie': 29


,review_title,review_text
id,,
439,Love. this. jacket.,I live in los angeles and this is the perfect ...
2481,Soft and super comfy,This hoodie is just what i had hoped it would ...
3087,Size up,"I love the ruching on this hoodie, which is wh..."
3100,"Soft, body conforming and comfy!",First i must say that i bought this zip up hoo...
5218,Really comfortable and cute - extremely soft f...,"This is probably the softest, most flattering ..."


Next, we define a function to return the most frequent category of all reviews that contain a given search term:

In [11]:
def top_category(df: pd.DataFrame, ID: int) -> list:
    """Get the most frequent category hierarchy (division, department and class) for a given product ID.

    Args:
        df: A Pandas dataframe with the preprocessed reviews data.
        ID: The product ID

    Returns:
        A dictionary with two key-value pairs:
            'category': The category hierarchy as a list [division, department, class].
            'sample_size': The number of similar reviews.
            'keyword': The keyword used to search the table for similar reviews.
    """
    # Search terms
    KEYWORDS = {
        72: "socks",
        492: "hoodie",
        152: "leg warmer",
        184: "tights",
        772: "sweatshirt",
        665: "underwear",
        136: "socks",
    }

    # Product category list
    CATEGORIES = category_list(df)
    
    # Get similar reviews based on keywords in review title and body text
    keyword = KEYWORDS[ID]
    similar_reviews = search(df, keyword)
    n_similar = len(similar_reviews)

    # Get the most frequent hierarchy using a pivot table
    pivot = similar_reviews\
        .filter(CATEGORIES)\
        .pivot_table(index=CATEGORIES, aggfunc='size')\
        .sort_values(ascending = False)
    levels = pivot.index.levels  # category levels 
    codes = pivot.index.codes  # integer codes mapping to category levels 
    top_code = [x[0] for x in codes]  
    top_level = [levels[i][top_code[i]] for i in range(len(top_code))] 
    return {'category': top_level, 'sample_size': n_similar, 'keyword': keyword}

In [12]:
# Example application
for ID in uncat_products:
    cat = top_category(reviews, ID)
    print("Category for product {i} from {n} samples containing '{w}': {c}"\
          .format(i = ID, n = cat['sample_size'], w = cat['keyword'], c = cat['category']))

Category for product 72 from 38 samples containing 'socks': ['intimates', 'intimate', 'legwear']
Category for product 492 from 29 samples containing 'hoodie': ['intimates', 'intimate', 'lounge']
Category for product 152 from 4 samples containing 'leg warmer': ['intimates', 'intimate', 'legwear']
Category for product 184 from 247 samples containing 'tights': ['general petite', 'dresses', 'dresses']
Category for product 772 from 149 samples containing 'sweatshirt': ['general', 'tops', 'knits']
Category for product 665 from 35 samples containing 'underwear': ['intimates', 'intimate', 'intimates']
Category for product 136 from 38 samples containing 'socks': ['intimates', 'intimate', 'legwear']


The suggested categories appear reasonable, so we'll add them to the uncategorised reviews:

In [13]:
def impute_categories(df: pd.DataFrame) -> pd.DataFrame:
          
    # Get products without category
    missing = list(uncategorised(df).index)
    
    if not missing:
        print("No uncategorised reviews.")
        return df
    else:
        # Get category hierarchy and IDs of missing products
        CATEGORIES = category_list(df)
        missing_products = uncategorised(df).product_id.unique()

        # Impute categories for missing products
        imputed = list(zip(*[top_category(df = df, ID = i)['category'] for i in missing_products]))
        
        # Create lookup table for imputed values
        imputed_lookup = pd.DataFrame(
            {CATEGORIES[i]: imputed[i] for i in range(len(imputed))}, 
            index = missing_products
        )
        
        # Copy the table and add imputed categories
        df_i = df.copy(deep = True)
        for cat in CATEGORIES:
            df_i.loc[missing, cat] = df.product_id.map(imputed_lookup[cat])
        
        return df_i

In [14]:
# Check that the imputation has removed null categories
reviews_imputed = impute_categories(reviews_clean)
reviews_imputed[category_list(reviews)].notnull().all()

product_category_division      True
product_category_department    True
product_category_class         True
dtype: bool

In [15]:
# Check imputed categories
reviews_imputed.reindex(uncategorised(reviews_clean).index)[category_list(reviews_imputed)]

,product_category_division,product_category_department,product_category_class
id,,,
9444,intimates,intimate,legwear
13767,intimates,intimate,lounge
13768,intimates,intimate,lounge
16216,intimates,intimate,legwear
16221,intimates,intimate,legwear
16223,intimates,intimate,legwear
18626,general petite,dresses,dresses
18671,general petite,dresses,dresses
20088,general,tops,knits


In [16]:
# Check overall table
summarise(reviews_imputed)

,dtype,n_null,n_valid,unique,mean,std,min,25%,50%,75%,max,skew
product_id,int64,0,22642,1179,919.340,202.266,1.0,861.0,936.0,1078.0,1205.0,-2.091
author_age,int64,0,22642,77,43.280,12.327,18.0,34.0,41.0,52.0,99.0,0.516
review_title,object,2966,19676,13994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
review_text,object,1,22641,22635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
star_rating,int64,0,22642,5,4.184,1.116,1.0,4.0,5.0,5.0,5.0,-1.288
recommend_flag,int64,0,22642,2,0.819,0.385,0.0,1.0,1.0,1.0,1.0,-1.656
upvotes,int64,0,22642,82,2.630,5.786,0.0,0.0,1.0,3.0,122.0,6.381
product_category_division,category,0,22642,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_category_department,category,0,22642,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_category_class,category,0,22642,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Check kedro implementation
%reload_kedro
from master.nodes.ETL import clean_missing_values
summarise(clean_missing_values(reviews))

2019-07-06 07:20:54,893 - root - INFO - ** Kedro project TWA Assignment
2019-07-06 07:20:54,895 - anyconfig - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/TWA/MBUSA_Text_Web_Analytics/conf/base/logging.yml
2019-07-06 07:20:54,908 - anyconfig - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/TWA/MBUSA_Text_Web_Analytics/conf/base/catalog.yml
2019-07-06 07:20:54,916 - anyconfig - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/TWA/MBUSA_Text_Web_Analytics/conf/base/credentials.yml
2019-07-06 07:20:54,920 - anyconfig - INFO - Loading: /Users/tomduke/Documents/Academic/MBusA/Modules/M3 - Advanced Analytics/TWA/MBUSA_Text_Web_Analytics/conf/base/parameters.yml
2019-07-06 07:20:54,924 - root - INFO - Defined global variables proj_dir, proj_name, conf and io


,dtype,n_null,n_valid,unique,mean,std,min,25%,50%,75%,max,skew
product_id,int64,0,22642,1179,919.340,202.266,1.0,861.0,936.0,1078.0,1205.0,-2.091
author_age,int64,0,22642,77,43.280,12.327,18.0,34.0,41.0,52.0,99.0,0.516
review_title,object,2966,19676,13994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
review_text,object,1,22641,22635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
star_rating,int64,0,22642,5,4.184,1.116,1.0,4.0,5.0,5.0,5.0,-1.288
recommend_flag,int64,0,22642,2,0.819,0.385,0.0,1.0,1.0,1.0,1.0,-1.656
upvotes,int64,0,22642,82,2.630,5.786,0.0,0.0,1.0,3.0,122.0,6.381
product_category_division,category,0,22642,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_category_department,category,0,22642,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_category_class,category,0,22642,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From the output above, we see that the imputation has removed the missing values for the product categories.